In [1]:
#!pip install shapely

In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import healpy as hp

from rubin_sim.data import get_data_dir
import sqlite3


from part1 import pointToLineDistance
from utils import create_constellation, starlink_constellation

In [27]:
# find the baseline survey simulation file that got downloaded with rubin_sim
dd = get_data_dir()
baseline_file = os.path.join(dd,'sim_baseline/baseline.db')

# Conenct to the sqlite database
con = sqlite3.connect(baseline_file)

# We can just load the whole thing into a dataframe
df = pd.read_sql('select * from observations;', con)

con.close()

In [28]:
# take a look at what the data looks like
df

,observationId,fieldRA,fieldDec,observationStartMJD,flush_by_mjd,visitExposureTime,filter,rotSkyPos,rotSkyPos_desired,numExposures,...,sunAz,sunRA,sunDec,moonRA,moonDec,moonDistance,solarElong,moonPhase,cummTelAz,scripted_id
0,0,310.024480,-60.812928,60218.001806,60218.023576,30.0,y,-297.249225,-297.249225,2,...,255.593220,186.644048,-2.870827,27.609463,11.956111,94.490314,102.958651,87.407902,169.454444,0
1,1,310.601871,-63.561425,60218.002254,60218.023576,30.0,y,-297.708278,-297.708278,2,...,255.500445,186.644453,-2.871001,27.615338,11.959438,95.029204,101.743959,87.404494,170.502875,0
2,2,311.292611,-66.317774,60218.002703,60218.023576,30.0,y,-297.909620,-297.909620,2,...,255.407493,186.644858,-2.871176,27.621208,11.962765,95.563446,100.497860,87.401088,171.406738,0
3,3,312.140731,-69.082666,60218.003152,60218.023576,30.0,y,-297.838337,-297.838337,2,...,255.314364,186.645264,-2.871350,27.627073,11.966093,96.092842,99.221261,87.397685,172.197791,0
4,4,304.170163,-73.375442,60218.003623,60218.023576,30.0,y,-309.290623,-309.290623,2,...,255.216260,186.645690,-2.871533,27.633232,11.969593,99.605370,94.821142,87.394110,177.239803,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081744,2081744,347.124408,-44.121665,63870.104312,63870.135744,30.0,u,16.313879,16.313879,1,...,226.952534,186.231318,-2.693059,256.910781,-17.679803,77.943326,130.154875,39.323876,118.256160,0
2081745,2081745,344.049134,-45.895531,63870.104746,63870.135744,30.0,u,5.751050,5.751050,1,...,226.779699,186.231710,-2.693228,256.915974,-17.679424,75.449290,127.589799,39.326371,127.956145,0
2081746,2081746,340.774233,-47.593274,63870.105180,63870.135744,30.0,u,-4.958170,-4.958170,1,...,226.606027,186.232103,-2.693397,256.921178,-17.679043,72.960208,124.997243,39.328872,136.957654,0
2081747,2081747,337.288929,-49.202077,63870.105614,63870.135744,30.0,u,-15.452126,-15.452126,1,...,226.432028,186.232495,-2.693565,256.926379,-17.678664,70.481515,122.382915,39.331373,144.924885,0


In [29]:
data=df[['fieldRA','fieldDec','observationStartMJD']]

In [30]:
data

,fieldRA,fieldDec,observationStartMJD
0,310.024480,-60.812928,60218.001806
1,310.601871,-63.561425,60218.002254
2,311.292611,-66.317774,60218.002703
3,312.140731,-69.082666,60218.003152
4,304.170163,-73.375442,60218.003623
...,...,...,...
2081744,347.124408,-44.121665,63870.104312
2081745,344.049134,-45.895531,63870.104746
2081746,340.774233,-47.593274,63870.105180
2081747,337.288929,-49.202077,63870.105614


***random helper functions because it wont import properly


In [16]:

import numpy as np 
from shapely.geometry import LineString
from shapely.geometry import Point
from rubin_sim.utils import Site
import ephem
from rubin_sim.utils import _angularSeparation, _buildTree, xyz_angular_radius
from rubin_sim.scheduler.utils import read_fields
from astropy import time
from shapely import geometry

class Constellation(object):
    """
    Have a class to hold ephem satellite objects

    Parameters
    ----------
    sat_tle_list : list of str
        A list of satellite TLEs to be used
    tstep : float (5)
        The time step to use when computing satellite positions in an exposure
    """

    def __init__(self, sat_tle_list, alt_limit=30., fov=3.5, tstep=5., exptime=30., seed=42):
        np.random.seed(seed)
        self.sat_list = [ephem.readtle(tle.split('\n')[0], tle.split('\n')[1], tle.split('\n')[2]) for tle in sat_tle_list]
        self.alt_limit_rad = np.radians(alt_limit)
        self.fov_rad = np.radians(fov)
        self._make_observer()
        self._make_fields()
        self.tsteps = np.arange(0, exptime+tstep, tstep)/3600./24.  # to days

        self.radius = xyz_angular_radius(fov)

    def _make_fields(self):
        """
        Make tesselation of the sky
        """
        # RA and dec in radians
        fields = read_fields()

        # crop off so we only worry about things that are up
        good = np.where(fields['dec'] > (self.alt_limit_rad - self.fov_rad))[0]
        self.fields = fields[good]

        self.fields_empty = np.zeros(self.fields.size)

        # we'll use a single tessellation of alt az
        leafsize = 100
        self.tree = _buildTree(self.fields['RA'], self.fields['dec'], leafsize, scale=None)

    def _make_observer(self):
        telescope = Site(name='LSST')

        self.observer = ephem.Observer()
        self.observer.lat = telescope.latitude_rad
        self.observer.lon = telescope.longitude_rad
        self.observer.elevation = telescope.height

    def advance_epoch(self, advance=100):
        """
        Advance the epoch of all the satellites
        """

        # Because someone went and put a valueError where there should have been a warning
        # I prodly present the hackiest kludge of all time
        for sat in self.sat_list:
            sat._epoch += advance

    def set_epoch(self, mjd):
        for sat in self.sat_list:
            sat._epoch = mjd

    #self.update_mjd gives a bunch of positions 
    def update_mjd(self, mjd, indx=None):
        """
        mjd : float
            The MJD to advance the satellites to (days)
        indx : list-like of ints
            Only propigate a subset of satellites. 
        """
        self.active_indx = indx

        self.observer.date = ephem.date(time.Time(mjd, format='mjd').datetime)

        self.altitudes_rad = []
        self.azimuth_rad = []
        self.eclip = []
        if self.active_indx is None:
            indx = np.arange(len(self.sat_list))
        else:
            indx = self.active_indx
        for i in indx:
            sat = self.sat_list[i]
            try:
                sat.compute(self.observer)
            except ValueError:
                self.set_epoch(self.observer.date+np.random.uniform()*10)
                sat.compute(self.observer)
            self.altitudes_rad.append(sat.alt)
            self.azimuth_rad.append(sat.az)
            self.eclip.append(sat.eclipsed)

        self.altitudes_rad = np.array(self.altitudes_rad)
        self.azimuth_rad = np.array(self.azimuth_rad)
        self.eclip = np.array(self.eclip)
        # Keep track of the ones that are up and illuminated
        self.above_alt_limit = np.where((self.altitudes_rad >= self.alt_limit_rad) & (self.eclip == False))[0]
    


    def check_pointing(self, pointing_alt, pointing_az, mjd, exposure_time, fov_radius=1.75):
        """Calculates the length of satellite streaks in a pointing. 
        Parameters
        ----------
        Param1 : float 
            the altitude of the pointing (degrees).
        Param2 : float
            the azimuth of the pointing (degrees).
        Param3 : float
            the current mjd (days).
        Param4: float 
            the length of exposure (seconds).
        fov_radius : float (1.75)
            The radius of the field of view (degrees), default 1.75.

        Returns
        -------
        list
            list of streak length in the given pointing"""
        
        fov_radius = np.radians(fov_radius)
        pointing_alt=np.radians(pointing_alt)
        pointing_az=np.radians(pointing_az)
        exposure_time=exposure_time/86400
        streak_len=[]

        self.update_mjd(mjd)
        inAlt_list=self.altitudes_rad + 0
        inAz_list=self.azimuth_rad + 0
        
        self.update_mjd(mjd+exposure_time)
        finAlt_list=self.altitudes_rad + 0 
        finAz_list=self.azimuth_rad + 0
        # print(self.above_alt_limit)
        
        for index in self.above_alt_limit: 
            # print(index)
            elem_list=list(zip(inAlt_list, inAz_list, finAlt_list, finAz_list))[index]
            initial_alt=elem_list[0]
            initial_az=elem_list[1]
            end_alt=elem_list[2]
            end_az=elem_list[3]
        # for initial_alt, initial_az, end_alt, end_az in zip(inAlt_list, inAz_list,
        #                                                       finAlt_list, finAz_list):
        # for i in range(40):
            distance=pointToLineDistance(initial_alt, initial_az, end_alt, end_az, pointing_alt, pointing_az)
            # print(fov_radius,distance)
            if distance<fov_radius:
                print("HI")
                streak=calculate_length(initial_alt, initial_az, end_alt, end_az, pointing_alt, pointing_az, fov_radius)
                streak_len.append(streak)
        return streak_len
        
        # # vfunc = np.vectorize(check_pointing_help)
        # zipped_list=np.array(list(zip(inAlt_list, inAz_list, finAlt_list, finAz_list)))
        # # print(zipped_list)
        # # iterable = (check_pointing_help(x) for x in zipped_list)
        # # result=np.fromiter(iterable, float)
        # # result = vfunc(zipped_list)
        # return list(map(check_pointing_help,zipped_list))



# def check_pointing_help(x,fov_radius=1.75):
#     print(x)
#     fov_radius = np.radians(fov_radius)
#     initial_alt=x[0]
#     initial_az=x[1]
#     end_alt=x[2]
#     end_az=x[3]
#     # initial_alt, initial_az, end_alt, end_az=x
#     distance=pointToLineDistance(initial_alt, initial_az, end_alt, end_az, pointing_alt, pointing_az)
#     print(f"distance is {distance}")
#     if distance<fov_radius:
#         print(f"streak is {streak}")
#         streak=calculate_length(initial_alt, initial_az, end_alt, end_az, pointing_alt, pointing_az, fov_radius)
#         return streak



# XXX--hopefully this can be vectorized, we want to pass arrays of satellite positions and not loop over them
def calculate_length(initial_alt, initial_az, end_alt, end_az, pointing_alt, pointing_az, radius ):
    """Helper funciton for check_pointing. 
    calculate the length of a streak after projecting the locations of the satellite and the pointing onto 2D.
    Parameters
    ----------
    Param1 : float 
        the initial altitude of the satellite (degree)
    Param2 : float
        the initial azimuth of the satellite (degree)
    Param3 : float
        the end altitude of the satellite (degree)
    Param4: float 
        the end azimuth of the satellite (degree)
    Param5 : float
        the altitude of the pointing (degree)
    Param6: float 
        the azimuth of the pointing(degree)
    Param7 : float
        the radius of the pointing (degree)


    Returns
    -------
    float
        the length of the satellite streak in the pointing (not sure what the unit should be -- degrees? meters?)
    """
    #stsart location 
    x1,y1=gnomonic_project_toxy(initial_alt, initial_az, pointing_alt, pointing_az)
    #end location
    x2,y2=gnomonic_project_toxy(end_alt, end_az, pointing_alt, pointing_az)

    # create your two points
    point_1 = geometry.Point(x1, y1)
    point_2 = geometry.Point(x2, y2)
    

    #from https://stackoverflow.com/questions/30844482/what-is-most-efficient-way-to-find-the-intersection-of-a-line-and-a-circle-in-py
    p = Point(0, 0)
    circle = p.buffer(radius).boundary
    circle_buffer=p.buffer(radius)
    line = LineString([(x1,y1), (x2,y2)])
    intersection = circle.intersection(line)
    try:
        if circle_buffer.contains(point_1) and circle_buffer.contains(point_2): 
            len=np.sqrt((x1-x2)**2+(y1-y2)**2)
            return len 
        elif circle_buffer.contains(point_1):
            x_2=intersection.coords[0][0]
            y_2=intersection.coords[0][1]
            len=np.sqrt((x1-x_2)**2+(y1-y_2)**2)
            return len 
        elif circle_buffer.contains(point_2):
            x_1=intersection.coords[0][0]
            y_1=intersection.coords[0][1]
            len=np.sqrt((x_1-x2)**2+(y_1-y2)**2) 
            return len  
        else:  
            p1=intersection.geoms[0].coords[0]
            p2=intersection.geoms[1].coords[0]
            len=np.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)
            return len 
    except:
            print(f"edge case: line: {line}, radius: {radius}, intersection: {intersection}")
            pass







#need to project start point, end point, center, and radius 
def gnomonic_project_toxy(RA1, Dec1, RAcen, Deccen):
    """Calculate x/y projection of RA1/Dec1 in system with center at RAcen, Deccen.
    Input radians. Grabbed from sims_selfcal
    Parameters
    ----------
    Param1 : float 
        the right ascension of the object (degrees)
    Param2 : float
        the declination of the object (degrees)
    Param3 : float
        the right ascension of the center of the system (degrees)
    Param4: float 
        the declination of the center of the system (degrees)



    Returns
    -------
    list
        two element list that contains the x,y projection of the object 
    
    
    """
    # also used in Global Telescope Network website
    cosc = np.sin(Deccen) * np.sin(Dec1) + np.cos(Deccen) * np.cos(Dec1) * np.cos(
        RA1 - RAcen
    )
    x = np.cos(Dec1) * np.sin(RA1 - RAcen) / cosc
    y = (
        np.cos(Deccen) * np.sin(Dec1)
        - np.sin(Deccen) * np.cos(Dec1) * np.cos(RA1 - RAcen)
    ) / cosc
    return x, y

In [17]:
tle_list=starlink_constellation()

In [18]:
len(tle_list)

11927

In [19]:
test_const=Constellation(tle_list)

#df is in degree 

In [34]:
res=[]
for i in range(20000): 
    exposure=30
    pointingRa=data.loc[i,'fieldRA']
    pointingDec=data.loc[i,'fieldDec']
    mjd=data.loc[i,'observationStartMJD']
    satlist=test_const.check_pointing(pointingRa,pointingDec, mjd, 30)
    res+=satlist
    if i%100==0:
        print(i)
print(res) 


0
100
200
300
400
500
600
700
HI
800
HI
HI
900
1000
1100
1200
1300
1400
1500
1600
HI
HI
HI
HI
edge case: line: LINESTRING (-0.0154940494053257 -0.8198073156271884, -0.0006060310333958 -1.6581680924560034), radius: 0.030543261909900768, intersection: LINESTRING EMPTY
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
1700
HI
HI
HI
HI
HI
HI
HI
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
HI
HI
HI
2800
2900
3000
3100
3200
3300
3400
3500
HI
3600
HI
HI
HI
HI
3700
3800
3900
4000
4100
4200
4300
HI
HI
HI
HI
4400
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
4500
4600
4700
4800
4900
5000
5100
5200
HI
HI
HI
HI
HI
HI
HI
HI
HI
edge case: line: LINESTRING (0.0212343558041212 0.0471391545744372, 0.242074632696771 0.9037039799930316), radius: 0.030543261909900768, intersection: LINESTRING EMPTY
HI
HI
HI
edge case: line: LINESTRING (0.0109315202056838 0.2266596503124159, 0.1937794189925606 1.295340139934285), radius: 0.030543261909900768, intersection: LINESTRING EMPTY
HI
5300
5400
550

In [25]:
data.loc[0,'fieldRA']

310.02448010095446

In [153]:
satlist=test_const.check_pointing(310.024480,-60.812928, 60218.004959, 30)

In [154]:
satlist

[]

In [117]:
list(filter(None, satlist))

[]

In [11]:
# OK, let's say we have a pointing at Alt,Az= 0,45. Let's test the calculate_length for various cases

In [12]:
# I think this is expecting all radians
radius = np.radians(1.75)
pointing_az = 0.
pointing_alt = np.radians(45.)

sat_init_alt = np.radians(20.)
sat_final_alt = np.radians(60.)

sat_init_az = 0.
sat_final_az = 0.

test_val = calculate_length(sat_init_alt, sat_init_az, sat_final_alt, sat_final_az,
                            pointing_alt, pointing_az, radius=radius)
print(np.degrees(test_val))

3.5


In [13]:
# OK, that worked! the streak did indeed go all the way across the field of view


In [14]:
# try another.
radius = np.radians(1.75)
pointing_az = np.radians(180.)
pointing_alt = np.radians(45.)

sat_init_alt = np.radians(45.)
sat_final_alt = np.radians(45.)

sat_init_az = np.radians(110.)
sat_final_az = np.radians(220.)

test_val = calculate_length(sat_init_alt, sat_init_az, sat_final_alt, sat_final_az,
                            pointing_alt, pointing_az, radius=radius)
if test_val is not None:
    print(np.degrees(test_val))
else:
    print('hmmm')

3.5


In [15]:
# OK, again, we got a satellite to cross the entire diameter correctly

# If I move the satellite alt up a bit, I should get a smaller value because I'm not crossing the center
radius = np.radians(1.75)
pointing_az = np.radians(180.)
pointing_alt = np.radians(45.)

sat_init_alt = np.radians(46.5)
sat_final_alt = np.radians(46.5)

sat_init_az = np.radians(110.)
sat_final_az = np.radians(220.)

test_val = calculate_length(sat_init_alt, sat_init_az, sat_final_alt, sat_final_az,
                            pointing_alt, pointing_az, radius=radius)
if test_val is not None:
    print(np.degrees(test_val))
else:
    print('hmmm')

1.793435724659027


In [16]:
# Victory, that's a smaller arc length

In [23]:
# Now let's break some things
radius = np.radians(1.75)
pointing_az = 0.
pointing_alt = np.radians(45.)

sat_init_alt = np.radians(45.)
sat_final_alt = np.radians(60.)

sat_init_az = 0.
sat_final_az = 0.

test_val = calculate_length(sat_init_alt, sat_init_az, sat_final_alt, sat_final_az,
                            pointing_alt, pointing_az, radius=radius)
if test_val is not None:
    print(np.degrees(test_val))
else:
    print('hmmm')

1.75


In [18]:
# Here the satellite starts inside the FoV and moves out. So that's a case that should work but didn't

In [19]:
# Test a wrap-around case. 
radius = np.radians(1.75)
pointing_az = 0.
pointing_alt = np.radians(45.)

sat_init_alt = np.radians(45.)
sat_final_alt = np.radians(45)

sat_init_az = np.radians(350.)
sat_final_az = np.radians(20.)

test_val = calculate_length(sat_init_alt, sat_init_az, sat_final_alt, sat_final_az,
                            pointing_alt, pointing_az, radius=radius)
if test_val is not None:
    print(np.degrees(test_val))
else:
    print('hmmm')

3.5


In [20]:
# oh good, that worked